In [12]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import sys
sys.path.append("../scripts")

import data
import models
import utils

In [13]:
X_train, y_train, X_val, y_val, X_test, y_test = data.get_fold("german", 0)
A_train = X_train.Gender == "Male"
A_val = X_val.Gender == "Male"
A_test = X_test.Gender == "Male"

col_trans = ColumnTransformer(
    [
        ("numeric", StandardScaler(), data.NUM_FEATURES["german"]),
        (
            "categorical",
            OneHotEncoder(
                drop="if_binary", sparse_output=False, handle_unknown="ignore"
            ),
            data.CAT_FEATURES["german"],
        ),
    ],
    verbose_feature_names_out=False,
)
col_trans.set_output(transform="pandas")
preprocess = Pipeline([("preprocess", col_trans)])
preprocess.fit(X_train)
X_train = preprocess.transform(X_train)
X_val = preprocess.transform(X_val)
X_test = preprocess.transform(X_test)

In [17]:
model = models.M2FGB()
model.fit(X_train, y_train, A_train);
y_test_pred = model.predict(X_test)

In [21]:
utils.equal_opportunity_score(y_test, y_test_pred, A_test)

0.12